In [2]:
%%writefile 'Environments/create-environment.yml'
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: transferlearning-pytorch
version: 1
image: mcr.microsoft.com/azureml/curated/pytorch-1.10-ubuntu18.04-py38-cuda11-gpu:36
description: Training enviroment PyTorch 1.10 / CUDA 11 / GPU

Writing Environments/create-environment.yml


In [3]:
!az config set defaults.workspace="testWorkspace"
!az config set defaults.group="personalFree"     
!az ml environment create --file Environments/create-environment.yml

Command group 'config' is experimental and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Command group 'config' is experimental and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "creation_context": {
    "created_at": "2024-06-22T19:24:32.698854+00:00",
    "created_by": "Jinay Bhavsar",
    "created_by_type": "User",
    "last_modified_at": "2024-06-22T19:24:32.698854+00:00",
    "last_modified_by": "Jinay Bhavsar",
    "last_modified_by_type": "User"
  },
  "description": "Training enviroment PyTorch 1.10 / CUDA 11 / GPU",
  "id": "azureml:/subscriptions/22419d70-5fbf-470c-8901-9f39256512ed/resourceGroups/personalFree/providers/Microsoft.MachineLearningServices/workspaces/testWorkspace/environments/transferlearning-pytorch/versions/1",
  "image": "mcr.microsoft.com/azureml/curated/pytorch-1.10-ubuntu18.04-py38-cuda11-gpu:36",
  "name": "transferlearning-pytorch",
  "os_type": "linux",
  "resourceGroup": "personalFree

In [35]:
%%writefile 'Components/train.yml'
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
type: command

name: pytorch_train_model
display_name: classification - Train model
version: 1.2

inputs:
  training_data:
    type: path
  epochs:
    type: integer
    default: 8
  learning_rate:
    type: number
    default: 0.001
  momentum: 
    type: number
    default: 0.9
  tag: 
    type: string
    default: "pyTorch"

outputs:
  train_output:
    type: path

code: ./src

environment: azureml:transferlearning-pytorch:1

command: python train.py --data_path ${{inputs.training_data}} --model_output_path ${{outputs.train_output}} --num_epochs ${{inputs.epochs}} --learning_rate ${{inputs.learning_rate}} --momentum ${{inputs.momentum}} --tag ${{inputs.tag}} 


Overwriting Components/train.yml


In [36]:
!az ml component create --file Components/train.yml

{
  "$schema": "https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json",
  "code": "azureml:/subscriptions/22419d70-5fbf-470c-8901-9f39256512ed/resourceGroups/personalFree/providers/Microsoft.MachineLearningServices/workspaces/testWorkspace/codes/249412e4-b434-4a03-958a-179c52fab0ac/versions/1",
  "command": "python train.py --data_path ${{inputs.training_data}} --model_output_path ${{outputs.train_output}} --num_epochs ${{inputs.epochs}} --learning_rate ${{inputs.learning_rate}} --momentum ${{inputs.momentum}} --tag ${{inputs.tag}}",
  "creation_context": {
    "created_at": "2024-06-22T20:27:41.139386+00:00",
    "created_by": "Jinay Bhavsar",
    "created_by_type": "User",
    "last_modified_at": "2024-06-22T20:27:41.271077+00:00",
    "last_modified_by": "Jinay Bhavsar",
    "last_modified_by_type": "User"
  },
  "display_name": "classification - Train model",
  "environment": "azureml:/subscriptions/22419d70-5fbf-470c-8901-9f39256512ed/resourceGroups/personalFree

In [31]:
%%writefile 'Components/register.yml'
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
type: command

name: pytorch_register_model
display_name: Register pyTorch model after run
version: 1.1

inputs:
  model_assets_path:
    type: path
  model_name:
    type: string
    default: "marvel-classification"
  model_file_name:
    type: string
    default: "model.pth"
  label_file_name:
    type: string
    default: "labels.txt" 

code: ./src

environment: azureml:transferlearning-pytorch:1

command: python register.py --model_assets_path ${{inputs.model_assets_path}} --model_name ${{inputs.model_name}} --model_file_name ${{inputs.model_file_name}} --label_file_name ${{inputs.label_file_name}}


Overwriting Components/register.yml


In [32]:
!az ml component create --file Components/register.yml

{
  "$schema": "https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json",
  "code": "azureml:/subscriptions/22419d70-5fbf-470c-8901-9f39256512ed/resourceGroups/personalFree/providers/Microsoft.MachineLearningServices/workspaces/testWorkspace/codes/249412e4-b434-4a03-958a-179c52fab0ac/versions/1",
  "command": "python register.py --model_assets_path ${{inputs.model_assets_path}} --model_name ${{inputs.model_name}} --model_file_name ${{inputs.model_file_name}} --label_file_name ${{inputs.label_file_name}}",
  "creation_context": {
    "created_at": "2024-06-22T20:24:46.443079+00:00",
    "created_by": "Jinay Bhavsar",
    "created_by_type": "User",
    "last_modified_at": "2024-06-22T20:24:46.536501+00:00",
    "last_modified_by": "Jinay Bhavsar",
    "last_modified_by_type": "User"
  },
  "display_name": "Register pyTorch model after run",
  "environment": "azureml:/subscriptions/22419d70-5fbf-470c-8901-9f39256512ed/resourceGroups/personalFree/providers/Microsoft.Machin

In [39]:
%%writefile 'Pipeline/trainings_pipeline.yml'
$schema: https://azuremlschemas.azureedge.net/latest/pipelineJob.schema.json
type: pipeline
experiment_name: pyTorch-Pipeline
description: Pipeline to create a Lego Classifier in PyTorch
display_name: Pipeline - Test

settings:
  default_datastore: azureml:workspaceartifactstore
  default_compute: azureml:testPytorch

inputs:
  training_data:
    mode: ro_mount
    path: azureml:LegoCharsV2:1

jobs:
  train_model:
    type: command
    component: azureml:pytorch_train_model:1.2
    inputs:
      training_data: ${{parent.inputs.training_data}}
      epochs: 8
      tag: "Train-1"
    outputs:
      train_output: 
        mode: upload
      
  register_pytorch:
    type: command
    component: azureml:pytorch_register_model:1.1
    inputs:
      model_assets_path: ${{parent.jobs.train_model.outputs.train_output}}
      model_name: "lego-pytorch-classification"
      model_file_name: "model.pth"

Overwriting Pipeline/trainings_pipeline.yml


In [40]:
!az ml job create --file Pipeline/trainings_pipeline.yml

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
{
  "creation_context": {
    "created

In [14]:
%%writefile 'Deployment/deployment.yml'
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
endpoint_name: testPytorch
name: pipeline-lego-pytorch-1-
app_insights_enabled: true
model: azureml:lego-pytorch-classification:1
code_configuration:
  code: ./
  scoring_script: score.py
environment: azureml:AzureML-pytorch-1.7-ubuntu18.04-py37-cpu-inference:32
instance_type: Standard_F8s_v2
instance_count: 1

Overwriting Deployment/deployment.yml


In [15]:
!az ml online-deployment create --file Deployment/deployment.yml

Check: endpoint testPytorch exists
(ResourceNotFound) The Resource 'Microsoft.MachineLearningServices/workspaces/testWorkspace/onlineEndpoints/testPytorch' under resource group 'personalFree' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Microsoft.MachineLearningServices/workspaces/testWorkspace/onlineEndpoints/testPytorch' under resource group 'personalFree' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix


In [41]:
%%writefile 'test-images.txt'
https://github.com/jbhavsar27/mlops-pytorch-azure-pipelines/blob/1300873c6c24244ff902673ca4d6c8130b3db0fd/prototyping/test-images/005.jpg
https://github.com/jbhavsar27/mlops-pytorch-azure-pipelines/blob/1300873c6c24244ff902673ca4d6c8130b3db0fd/prototyping/test-images/007.jpg
https://github.com/jbhavsar27/mlops-pytorch-azure-pipelines/blob/1300873c6c24244ff902673ca4d6c8130b3db0fd/prototyping/test-images/009.jpg
https://github.com/jbhavsar27/mlops-pytorch-azure-pipelines/blob/1300873c6c24244ff902673ca4d6c8130b3db0fd/prototyping/test-images/012.jpg
https://github.com/jbhavsar27/mlops-pytorch-azure-pipelines/blob/1300873c6c24244ff902673ca4d6c8130b3db0fd/prototyping/test-images/031.jpg
https://github.com/jbhavsar27/mlops-pytorch-azure-pipelines/blob/1300873c6c24244ff902673ca4d6c8130b3db0fd/prototyping/test-images/032.jpg
https://github.com/jbhavsar27/mlops-pytorch-azure-pipelines/blob/1300873c6c24244ff902673ca4d6c8130b3db0fd/prototyping/test-images/033.jpg
https://github.com/jbhavsar27/mlops-pytorch-azure-pipelines/blob/1300873c6c24244ff902673ca4d6c8130b3db0fd/prototyping/test-images/040.jpg
https://github.com/jbhavsar27/mlops-pytorch-azure-pipelines/blob/1300873c6c24244ff902673ca4d6c8130b3db0fd/prototyping/test-images/041.jpg

Writing test-images.txt
